# ⚡ TFIM: Quantum Phase Transition

Study the transverse-field Ising model phase transition at the critical point g=1.

In [ ]:
!rm -rf PyTenNet && git clone https://github.com/tigantic/PyTenNet.git
import sys; sys.path.insert(0, 'PyTenNet')

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tensornet import MPS, tfim_mpo, dmrg

torch.manual_seed(42)
print(f'PyTorch {torch.__version__}')

## 1. Energy vs Transverse Field

In [ ]:
L = 20
g_values = np.linspace(0.2, 2.0, 19)
energies = []

for g in g_values:
    H = tfim_mpo(L=L, J=1.0, g=g)
    psi = MPS.random(L=L, d=2, chi=64)
    psi, E, _ = dmrg(psi, H, num_sweeps=15, chi_max=64)
    energies.append(E/L)
    print(f'g={g:.2f}: E/L = {E/L:.6f}')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(g_values, energies, 'o-', linewidth=2, markersize=6, color='navy')
plt.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Critical point g=1')
plt.fill_betweenx(plt.ylim(), 0, 1, alpha=0.1, color='blue', label='Ordered phase')
plt.fill_betweenx(plt.ylim(), 1, 2, alpha=0.1, color='green', label='Disordered phase')
plt.xlabel('Transverse field g', fontsize=12)
plt.ylabel('Energy per site E/L', fontsize=12)
plt.title(f'TFIM Phase Diagram (L={L})', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Entanglement Entropy at Criticality

In [ ]:
# Compare entanglement at g=0.5 (ordered), g=1.0 (critical), g=2.0 (disordered)
L = 30
g_list = [0.5, 1.0, 2.0]
labels = ['g=0.5 (ordered)', 'g=1.0 (critical)', 'g=2.0 (disordered)']
colors = ['blue', 'red', 'green']

plt.figure(figsize=(10, 6))

for g, label, color in zip(g_list, labels, colors):
    H = tfim_mpo(L=L, J=1.0, g=g)
    psi = MPS.random(L=L, d=2, chi=64)
    psi, E, _ = dmrg(psi, H, num_sweeps=15, chi_max=64)
    
    entropies = [psi.entropy(bond).item() for bond in range(L-1)]
    plt.plot(range(1, L), entropies, 'o-', label=label, color=color, markersize=4)
    print(f'{label}: E/L = {E/L:.6f}, S_max = {max(entropies):.4f}')

plt.xlabel('Bond position', fontsize=12)
plt.ylabel('Entanglement entropy', fontsize=12)
plt.title(f'Entanglement Profiles (L={L})', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Central Entropy vs g (Order Parameter)

In [ ]:
L = 20
g_values = np.linspace(0.2, 2.0, 25)
central_entropy = []

for g in g_values:
    H = tfim_mpo(L=L, J=1.0, g=g)
    psi = MPS.random(L=L, d=2, chi=64)
    psi, E, _ = dmrg(psi, H, num_sweeps=12, chi_max=64)
    S_center = psi.entropy(L//2 - 1).item()
    central_entropy.append(S_center)
    print(f'g={g:.2f}: S_center = {S_center:.4f}')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(g_values, central_entropy, 'o-', linewidth=2, markersize=6, color='purple')
plt.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Critical point')
plt.xlabel('Transverse field g', fontsize=12)
plt.ylabel('Central entanglement entropy', fontsize=12)
plt.title('Entropy Peak at Quantum Critical Point', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find the peak
peak_idx = np.argmax(central_entropy)
print(f'
Peak entropy at g = {g_values[peak_idx]:.2f}')

## 4. Finite-Size Scaling at Criticality

In [ ]:
# At critical point, S ~ (c/6) * log(L) for central entropy
L_values = [8, 12, 16, 20, 24, 30, 40]
S_center_values = []

for L in L_values:
    H = tfim_mpo(L=L, J=1.0, g=1.0)  # Critical point
    psi = MPS.random(L=L, d=2, chi=64)
    psi, E, _ = dmrg(psi, H, num_sweeps=15, chi_max=64)
    S = psi.entropy(L//2 - 1).item()
    S_center_values.append(S)
    print(f'L={L:2d}: S_center = {S:.4f}')

In [ ]:
# Fit S = (c/6) * log(L) + const
log_L = np.log(L_values)
coeffs = np.polyfit(log_L, S_center_values, 1)
c_fit = coeffs[0] * 6  # Central charge

plt.figure(figsize=(8, 6))
plt.plot(log_L, S_center_values, 'o', markersize=10, label='DMRG data')
plt.plot(log_L, np.polyval(coeffs, log_L), '--', linewidth=2, 
         label=f'Fit: c = {c_fit:.3f}')
plt.xlabel('log(L)', fontsize=12)
plt.ylabel('Central entropy S', fontsize=12)
plt.title('Finite-Size Scaling at Critical Point', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f'Extracted central charge: c = {c_fit:.3f}')
print(f'Expected (Ising CFT): c = 0.5')